# Project fruit
The goal of this project is classifying fruits and determining its ripeness

In [8]:
import copy
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet50, vgg16

# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

In [14]:
fruits_dir = 'C:/Users/emanu/Documents/Programming/Python/Project/ADL/data/fruits-360_dataset/fruits-360/Training'

# define the transforms to be applied to each image
transform = transforms.Compose([
    transforms.ToTensor()
])
# Augmentation ideas:
# Rotation, Translation, randomCrop, normalize

fruits_dataset = datasets.ImageFolder(root=fruits_dir, transform=transform)

# Split up dataset into train, valid and test
# Visualize some samples?

BATCH_SIZE = 64
train_loader = DataLoader(fruits_dataset, batch_size=BATCH_SIZE, shuffle=True)

#####

model1 = resnet50(weights="IMAGENET1K_V2")
model2 = vgg16(pretrained=True)
# correct implementation?
# "The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead."
# I can't get the suggested "weights=VGG16_Weights.IMAGENET1K_V1" to work
# Use other depths of the networks?

#####

c:\Users\emanu\anaconda3\envs\nnlm\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\emanu\anaconda3\envs\nnlm\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):

    train_losses = []
    val_losses = []
    best_model_loss = 1
    
    for epoch in range(num_epochs):

        # Training
        for i, (data, labels) in enumerate(train_loader):
      
            prediction = model.forward(data)

            train_loss = criterion(prediction, labels)

            train_loss.backward()

            optimizer.step()

            optimizer.zero_grad()
            
        print(f'\rEpoch {epoch+1}, batch {i+1}/{len(train_loader)} - Loss: {train_loss}')

        train_losses.append(train_loss)
        # writer.add_scalar("Loss/train", train_loss, epoch)

        # Validation
        for i, (data, labels) in enumerate(val_loader):

            prediction = model.forward(data)

            val_loss = criterion(prediction, labels)

            val_losses.append(val_loss)

        print(f"loss validation: {val_loss}")

        # Keep the best model
        if val_losses[-1] < best_model_loss:
            best_model = copy.deepcopy(model)
            best_model_loss = val_losses[-1]

        # writer.add_scalar("Loss/validation", loss_val, epoch)

    print(f"\nBest model loss: {best_model_loss}")
    return best_model, train_losses, val_losses

def get_accuracy(network, loader):
    
    with torch.no_grad():
        correct = 0
        total = 0
        y_pred = []
        y_true = []

        for x, (data, labels) in enumerate(loader):

            prediction = network.forward(data)

            for i in range(len(data)):

                y_true.append(labels[i].item())
                y_pred.append(torch.argmax(prediction[i]).item())
                if y_true[i] == y_pred[i]:
                    correct += 1        
    
            total += float(len(data))
    
        score = correct/total

        accuracy = score

    return accuracy

In [ ]:
LEARNING_RATE = 1e-3
EPOCHS = 10

criterion = nn.CrossEntropyLoss()
optimizer1 = torch.optim.Adam(model1.parameters(), lr = LEARNING_RATE)
optimizer2 = torch.optim.Adam(model2.parameters(), lr = LEARNING_RATE)

# Training
trained_model1, train_loss1, val_loss1 = train_model(model1, criterion, optimizer1)
trained_model2, train_loss2, val_loss2 = train_model(model2, criterion, optimizer2)

# Testing
test_acc1 = get_accuracy(trained_model1)
test_acc2 = get_accuracy(trained_model2)

print(f"ResNet50 accuracy: {test_acc1*100}%, VGGNet16 accuracy: {test_acc2*100}%")
# writer.flush()